In [1]:
%load_ext autoreload
%autoreload 2

# Imports and definitions

In [2]:
import xarray as xr
xr.set_options(keep_attrs=True)

from datetime import datetime
from pathlib import Path

In [3]:
import ecephys_analyses as ea
from sglxarray import load_trigger
from ecephys import xrsig

In [4]:
def get_bandpower(sig, chans, f_range):
    fft_window_length = 4 # seconds
    nperseg = int(fft_window_length * sig.fs) 
    noverlap = nperseg // 4 # 1 second overlap
    spg = xrsig.parallel_spectrogram_welch(sig, nperseg=nperseg, noverlap=noverlap).median(dim="channel")
    return xrsig.get_bandpower(spg, f_range)

In [5]:
# This should probably not compute cx power when sr chans is absent. 

def get_condition_bandpower(subject, experiment, condition, probe):
    cx_chans = ea.get_channels(subject, experiment, probe, "superficial_ctx", asarray=False)
    wm_chans = ea.get_channels(subject, experiment, probe, "white_matter", asarray=False)
    
    bin_paths = ea.get_sglx_style_datapaths(subject, experiment, condition, "lf.bin")
    sr_chans_paths = ea.get_sglx_style_datapaths(subject, experiment, condition, "sr_chans.csv")
    power_paths = ea.get_sglx_style_datapaths(subject, experiment, condition, "bandpower.nc")
    
    for bin_path, sr_chans_path, power_path in zip(bin_paths, sr_chans_paths, power_paths):
        sr_chans_df = ea.load.load_sr_chans(sr_chans_path)
        epochs = list()
        for epoch in sr_chans_df.itertuples():
            if not epoch.sr_chans:
                continue
            cx = load_trigger(bin_path, cx_chans, start_time=epoch.start_time, end_time=epoch.end_time)
            wm = load_trigger(bin_path, wm_chans, start_time=epoch.start_time, end_time=epoch.end_time)
            sr = load_trigger(bin_path, epoch.sr_chans, start_time=epoch.start_time, end_time=epoch.end_time)

            cx_wm_ref = cx - wm.values
            cx_wm_ref.attrs['reference'] = wm.channel.values

            sr_wm_ref = sr - wm.values
            sr_wm_ref.attrs['reference'] = wm.channel.values

            delta = (0.5, 4)
            theta = (5, 10)
            power = xr.Dataset({'mpta_delta_wm_ref': get_bandpower(cx_wm_ref, cx_chans, delta),
                                'mpta_theta_wm_ref': get_bandpower(cx_wm_ref, cx_chans, theta),
                                'sr_delta_wm_ref': get_bandpower(sr_wm_ref, epoch.sr_chans, delta),
                                'sr_theta_wm_ref': get_bandpower(sr_wm_ref, epoch.sr_chans, theta)})
            epochs.append(power)
            
        if not epochs:
            continue
        power = xr.concat(epochs, dim="time")
        Path(power_path).parent.mkdir(parents=True, exist_ok=True) # Create parent directories if they do not already exist.
        power.to_netcdf(power_path)
        power.close()
        
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Finished {str(bin_path)}")

In [6]:
get_condition_bandpower("Luigi", "sleep-homeostasis", "all", "imec0")

nChan: 385, nFileSamp: 18000070
nChan: 385, nFileSamp: 18000070
nChan: 385, nFileSamp: 18000070
13:13:50: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX9-Luigi/6-20-2021/6-20-2021_g0/6-20-2021_g0_imec0/6-20-2021_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 18000070
nChan: 385, nFileSamp: 18000070
nChan: 385, nFileSamp: 18000070
13:20:30: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX9-Luigi/6-20-2021/6-20-2021_g0/6-20-2021_g0_imec0/6-20-2021_g0_t1.imec0.lf.bin
nChan: 385, nFileSamp: 18000070
nChan: 385, nFileSamp: 18000070
nChan: 385, nFileSamp: 18000070
13:27:16: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX9-Luigi/6-20-2021/6-20-2021_g0/6-20-2021_g0_imec0/6-20-2021_g0_t2.imec0.lf.bin
nChan: 385, nFileSamp: 18000070
nChan: 385, nFileSamp: 18000070
nChan: 385, nFileSamp: 18000070
13:34:10: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX9-Luigi/6-20-2021/6-20-2021_g0/6-20-2021_g0_imec0/6-20-2021_g0_t3.imec0.lf.bin
nChan: 385, nFileSamp: 18000070
nChan: 3